In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
# https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=184723
# https://www.cian.ru/cat.php?deal_type=sale&offer_type=flat&region=184723

In [4]:
base_url='https://www.cian.ru/cat.php'
req_param='deal_type=sale&offer_type=flat&region=184723'

In [5]:
from lib.parser import AdsListParser

In [6]:
class CianParser(AdsListParser):
    
    def __init__(self,driver):
        super().__init__(
            driver=driver, 
            base_url='https://www.cian.ru/cat.php',
            item_tag=['article',{'data-name':'CardComponent',},],
            paginator_url_param='p',
        )
        logging.info('CianParser: downloader init')
    
    def _is_last_page(self,root,p): 
        try:
            paginator = root.find('div',{'data-name':'Pagination',})
            pl = [ tag.text for tag in root.find_all('li') ][-1]
            return str(p)==pl
        except Exception as e:
            logging.warning(f'CianParser: parse pagination error: {e}')
            return True
        
    def _parse_item(self,tag): 
        return { 
             'OfferTitle': self._get_title(tag),
          'OfferSubtitle': self._get_subtitle(tag),
               'Deadline': self._get_deadline(tag),
              'MainPrice': self._get_price(tag),
              'PriceInfo': self._get_price_m(tag),
               'GeoLabel': self._get_adr(tag),
              'TimeLabel': self._get_ts( tag ),  
               'LinkArea': self._get_link( tag ),
            'Description': self._get_descr(tag),
                        
            }
    
    @staticmethod
    def _get_deadline(tag):
        try:
            return tag.find('span',{'data-mark':'Deadline',}).text
        except:
            return None
        
    @staticmethod
    def _get_title(tag):
        try:
            return tag.find('span',{'data-mark':'OfferTitle',}).text
        except:
            return None
        
    @staticmethod
    def _get_subtitle(tag):
        try:
            return tag.find('span',{'data-mark':'OfferSubtitle',}).text
        except:
            return None
        
    @staticmethod
    def _get_price(tag):
        try:
            return tag.find('span',{'data-mark':'MainPrice',}).text
        except:
            return None

    @staticmethod
    def _get_price_m(tag):
        try:
            return tag.find('p',{'data-mark':'PriceInfo',}).text
        except:
            return None
        
    @staticmethod
    def _get_adr(tag):
        try:
            return [ t.text for t in tag.find_all('a',{'data-name':'GeoLabel',}) ]
        except:
            return []
    
    @staticmethod
    def _get_ts(tag):
        try:
            tag_ = tag.find('div',{'data-name':'TimeLabel',})
            return [ t.text for t in tag_.find_all('span') ]
        except:
            return []
        
    @staticmethod
    def _get_link(tag):
        try:
            return tag.find('div',{'data-name':'LinkArea',}).find('a').attrs['href']
        except:
            return None
        
    @staticmethod
    def _get_descr(tag):
        try:
            return tag.find('div',{'data-name':'Description',}).text
        except:
            return None
    

---

In [7]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
from lib.downloader import DownloaderSeleniumFirefox

df,html = CianParser(
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param, keep_html=True, page_limit=3 )

print(len(df))

[INFO    ] 2022-09-07 13:21:24 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-07 13:21:24 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-07 13:21:26 | AdsListParser: downloader init
[INFO    ] 2022-09-07 13:21:26 | CianParser: downloader init
[INFO    ] 2022-09-07 13:21:26 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-07 13:21:29 | AdsListParser: read page 1
[WARNING ] 2022-09-07 13:21:29 | CianParser: parse pagination error: list index out of range
[INFO    ] 2022-09-07 13:21:29 | AdsListParser: last page detected
[INFO    ] 2022-09-07 13:21:29 | DownloaderSeleniumFirefox: close virtual browser
0


In [11]:
# html

In [ ]:
# from lib.downloader import DownloaderSimple
# df,html = CianParser(driver=DownloaderSimple()).load(req_param, keep_html=True, page_limit=3)
# # html = DownloaderSimple().get(url)

In [12]:
with open('tmp/cian.html','wt') as f: f.write(html[0])

In [ ]:
# df

In [ ]:
# html

---

In [ ]:
# from lib.downloader import DownloaderSimple
# html = DownloaderSimple().get(url)
# with open('tmp/cian_.html','wt') as f: f.write(html[0])

---

In [ ]:
# <article data-name="CardComponent" 

In [ ]:
# <span data-mark="OfferTitle"
# <span data-mark="OfferSubtitle" 
# <span data-mark="Deadline"

# <a data-name="GeoLabel"

# <span data-mark="MainPrice"

# <p data-mark="PriceInfo" 

# <div data-name="Description" 

# <div data-name="LinkArea" class="_93444fe79c--container--kZeLu _93444fe79c--link--DqDOy">
# <a href="https://sevastopol.cian.ru/sale/flat/273181085/" class="_93444fe79c--link--eoxce">

# <div data-name="TimeLabel" 

In [ ]:
# def get_link(tag):
#     return tag.find('a').attrs['href']

# def get_ts(tag):
#     return [ t.text for t in tag.find_all('span') ]

# def get_adr(tag):
#     return [ t.text for t in tag.find_all('a',{'data-name':'GeoLabel',}) ]

# def parse_item(tag):
#     return {
#     'OfferTitle':tag.find('span',{'data-mark':'OfferTitle',}).text,
    
#     #'OfferSubtitle': tag.find('span',{'data-mark':'OfferSubtitle',}).text,
#     # 'Deadline': tag.find('span',{'data-mark':'Deadline',}).text,
    
#     'GeoLabel':  get_adr(tag),  # tag.find('span',{'data-name':'GeoLabel',}).text,
        
#     'MainPrice': tag.find('span',{'data-mark':'MainPrice',}).text,
#     'PriceInfo': tag.find('p',{'data-mark':'PriceInfo',}).text,
#     'Description': tag.find('div',{'data-name':'Description',}).text,
#     'LinkArea': get_link( tag.find('div',{'data-name':'LinkArea',})), 
#     'TimeLabel': get_ts( tag.find('div',{'data-name':'TimeLabel',})),  
#     }
    
    

In [ ]:
# root = BeautifulSoup(html,'html.parser')
# text = [
#     parse_item(tag)
#     for tag in root.find_all('article',{'data-name':'CardComponent',}) 
# ]


In [ ]:
# pd.DataFrame(text)

In [ ]:
# text[0]

In [ ]:
# text[1]

----

In [ ]:
# # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
# class AvitoDownloader:
    
#     def __init__(self,driver, base_url='https://www.avito.ru'):
#         logging.info('AvitoDownloader: downloader init')
#         self._base_url = base_url
#         self._driver = driver # ссылка на открытый браузер
        
        
#     # загрузить список объявлений Авито
#     # из раздела url_ext ( 'sevastopol/kvartiry/prodam' )
#     # не более page_limit страниц (если неопределенно то все страницы)
#     def load(self, avito_path, page_limit=None, show_pbar=False, keep_html=False): 
#         html = [] # считанный "чистый" html
#         data = [] # данные извлечённые парсером из html
#         try:
#             if re.match('^http.*',avito_path):
#                 logging.warning('AvitoDownloader: incorrect avito_path')

#             url = self._base_url + '/' + avito_path + '?'
            
#             # читаем и парсим оставшиеся страницы списка объявлений (начиная со второй)
#             logging.info('AvitoDownloader: start read and parse pages...')

#             page,root,src = self._read_page(url) # читаем и парсим первую страницу списка объявлений
#             data.extend(page)
#             if keep_html: html.append(src)
                
#             # считываем количество страниц, на которые поделен список объявлений
#             npages = self._get_pages_count(root,page_limit=page_limit)
            
#             npages_ = tqdm(range(2,npages+1)) if show_pbar else range(2,npages+1)
#             for p in npages_: 
#                 # читаем и парсим страницу p списка объявлений
#                 page,_,src = self._read_page(url+f'&p={p}',npage=p) 
#                 data.extend(page)
#                 if keep_html: html.append(src)
                           
#         except Exception as e:
#             logging.error(e) # перехватываем и логируем описания возникших ошибок

#         finally: # завершение процесса чтения
#             data = pd.DataFrame(data).dropna()
#             data['ts']  = dtm.now()
#             # выдаём список полученных объявлений и их исходный html
#             return (data,html) if keep_html else data 
                 
          
#     # читаем страницу Авито по url
#     def _read_page(self,url,npage=1): 
#         html = self._driver.get(url)
#         root = BeautifulSoup(html,'html.parser')
#         return self._parse_page(root,npage=npage),root, html,  

#     @classmethod
#     def _parse_page(cls, root, npage):
#         return [ 
#             cls._parse_item(tag)|{'avito_page':npage,} 
#             for tag in root.find_all('div',{'data-marker':'item'}) 
#         ]
        
#     @staticmethod
#     def _parse_item(tag): 
#         return { 'avito_id': tag.attrs['data-item-id'], 'text':tag.text, } # { 'html':str(tag), }

#     @classmethod
#     def _get_pages_count(cls,root,page_limit):
#         pages = cls._parse_pages_count(root)
#         logging.info(f'{pages} pages for read')
#         if not(page_limit is None): 
#             pages = min(pages,page_limit+1)
#             logging.info(f'apply page limit - {pages} pages')
#         return pages
            
#     @staticmethod
#     def _parse_pages_count(root):
#         pp = re.sub( r'.*?p=', '', root.find_all('a',{'class':'pagination-page'})[-1].attrs['href'] ) 
#         return 1 if not re.match(r'\d{1,3}', pp) else int(pp)

In [ ]:
# # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
# class AvitoDownloaderRealty(AvitoDownloader):
    
#     @classmethod
#     def _parse_item(cls,tag):
#         return {    
#             'avito_id': tag.attrs['data-item-id'],   # https://www.avito.ru/<avito_id>
#             'title': cls._parse_item_tile(tag),
#             'price': cls._parse_item_price(tag),
#             'obj_name': cls._parse_item_dev_name(tag),
#             'adr': cls._parse_item_adr(tag),
#             'description': cls._parse_item_description(tag),
#             }
        
#     @staticmethod
#     def _parse_item_tile(tag):
#         try:
#             return tag.find('a',attrs={'itemprop':'url'}).attrs['title']
#         except:
#             return ''    
      
#     @staticmethod
#     def _parse_item_price(tag):
#         try:
#             return tag.find('meta',attrs={'itemprop':'price'}).attrs['content']
#         except:
#             return ''
        
#     @staticmethod
#     def _parse_item_dev_name(tag):
#         try:
#             return tag.find('div',{'data-marker':'item-development-name'}).text
#         except:
#             return '' 
           
#     @staticmethod
#     def _parse_item_adr(tag):
#         regex_address = re.compile('geo-address-.*')
#         try:
#             return tag.find('span',{'class':regex_address}).text
#         except:
#             return '' 
            
#     @staticmethod
#     def _parse_item_description(tag):
#         regex_address = re.compile('geo-address-.*')
#         try:
#             return tag.find('meta',attrs={'itemprop':'description'}).attrs['content']
#         except:
#             return ''        

In [ ]:
# class AdsListParser:
    
#     def __init__( self,driver, base_url, item_tag, paginator_url_param='p', ):
#         logging.info('AdsListParser: downloader init')
#         self._base_url = base_url
#         self._paginator_url_param = paginator_url_param
#         self._driver = driver
#         self._item_tag = item_tag # ['article',{'data-name':'CardComponent',}]

#     # загрузить список объявлений не более page_limit страниц 
#     def load(self, req_param, page_limit=200, keep_html=False): 
#         html = [] # считанный "чистый" html
#         data = [] # данные извлечённые парсером из html
#         try:
#             url = self._base_url + '?' + req_param
            
#             # читаем и парсим оставшиеся страницы списка объявлений (начиная со второй)
#             logging.info('AdsListParser: start read and parse pages...')

#             for p in range(1,page_limit):
#                 # читаем и парсим страницу p списка объявлений
#                 page,root,src = self._read_page(url+f'&{self._paginator_url_param}={p}',npage=p) 
#                 data.extend(page)
#                 if keep_html: html.append(src)
#                 logging.info(f'AdsListParser: read page {p}')
#                 if self._is_last_page(root,p): 
#                     logging.info('AdsListParser: last page detected')
#                     break
                                                
#         except Exception as e:
#             logging.error(e) # перехватываем и логируем описания возникших ошибок

#         finally: # завершение процесса чтения
#             data = pd.DataFrame(data).dropna()
#             data['ts']  = dtm.now()
#             # выдаём список полученных объявлений и их исходный html
#             return (data,html) if keep_html else data 
          
#     # читаем страницу по url
#     def _read_page(self,url,npage=1): 
#         html = self._driver.get(url)
#         root = BeautifulSoup(html,'html.parser')
#         return self._parse_page(root,npage=npage),root, html,  

#     def _parse_page(self, root, npage):
#         return [ 
#             self._parse_item(tag)|{'page':npage,} 
#             for tag in root.find_all( *self._item_tag )
#         ]
    
#     def _parse_item(self,tag): 
#         return { 'text': tag.text, }

#     def _is_last_page(self,root,p): 
#         return True